In [1]:
!pip install polars numpy implicit tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 45.8 MB/s eta 0:00:0000:0100:01


In [2]:
import polars as pl
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_matrix
import implicit

RANDOM_STATE = 42
N_PREDICTIONS = 100

In [3]:
train = pl.read_parquet("/kaggle/input/recsysdata/hh_recsys_train_hh.pq")
test = pl.read_parquet("/kaggle/input/recsysdata/hh_recsys_test_hh.pq")
# vacancies = pl.read_parquet('/content/drive/MyDrive/Colab Notebooks/RecSys Competition 2023 2024/hh_recsys_vacancies.pq')

In [5]:
train.head()

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_332060""","""s_28301374""","[""v_2571684"", ""v_488179"", … ""v_2633899""]","[2, 2, … 2]","[2023-11-01 00:40:58.105, 2023-11-01 00:58:13.091, … 2023-11-01 01:35:54.456]"
"""u_1057881""","""s_33868982""","[""v_665861""]",[2],[2023-11-01 00:23:51.452]
"""u_1036784""","""s_32474802""","[""v_2594840""]",[2],[2023-11-01 00:52:34.023]
"""u_786220""","""s_14060785""","[""v_1473781"", ""v_1622905"", … ""v_2394151""]","[1, 2, … 2]","[2023-11-01 00:58:20.793, 2023-11-01 01:17:04.774, … 2023-11-01 01:17:40.149]"
"""u_639152""","""s_23205986""","[""v_695738"", ""v_22433"", … ""v_1078110""]","[2, 1, … 2]","[2023-11-01 01:14:20.828, 2023-11-01 00:54:08.636, … 2023-11-01 01:00:53.660]"


In [6]:
predictions = test.select(pl.col('vacancy_id').list.unique(maintain_order=True).list.tail(2))['vacancy_id'].to_list()
df = pl.DataFrame({'predictions': [i for i in predictions]} )
test = test.with_columns(df)

In [8]:
аfor i in tqdm(range(1, 70)):
    new_df = pl.concat([new_df, train[(i-1)*50_000:i*50_000].explode(columns=["vacancy_id", "action_type", "action_dt"]).sort("action_dt")])
new_df.head()

100%|██████████| 69/69 [00:45<00:00,  1.51it/s]


user_id,session_id,vacancy_id,action_type,action_dt
str,str,str,i64,datetime[ns]
"""u_370846""","""s_24698241""","""v_697571""",1,2023-11-01 00:00:00.919
"""u_461521""","""s_7559925""","""v_2514797""",2,2023-11-01 00:00:06.973
"""u_332204""","""s_6570164""","""v_742810""",2,2023-11-01 00:00:12.594
"""u_229153""","""s_23936793""","""v_1411424""",2,2023-11-01 00:00:13.518
"""u_1128173""","""s_14266530""","""v_248154""",2,2023-11-01 00:00:16.114


In [9]:
train = new_df

In [10]:
train["action_type"].unique()

action_type
i64
1
2
3


In [9]:
test

user_id,session_id,vacancy_id,action_type,action_dt,predictions
str,str,list[str],list[i64],list[datetime[ns]],list[str]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]","[""v_2597196"", ""v_1223061""]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]","[""v_470400"", ""v_1530783""]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]","[""v_1592343"", ""v_1329494""]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]","[""v_1375331"", ""v_1922852""]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]","[""v_2152997"", ""v_1217630""]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]","[""v_698513"", ""v_212060""]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]","[""v_1828011"", ""v_2297339""]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]","[""v_206700"", ""v_528671""]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]","[""v_36093"", ""v_1500697""]"


# Simple ALS

In [10]:
unique_users = train['user_id'].unique().to_list()
unique_vacancies = train['vacancy_id'].explode().unique().to_list()

user2idx = {user_id: idx for idx, user_id in enumerate(unique_users)}
vac2idx = {vac_id: idx for idx, vac_id in enumerate(unique_vacancies)}

idx2vac = {idx: vac_id for vac_id, idx in vac2idx.items()}
action_weights = {
    1: 4.0,
    2: 1.0,
    3: 2.0
}

pairs = train.select(['user_id', 'vacancy_id', 'action_type'])
users = list(map(int,pairs['user_id'].replace(user2idx).to_numpy()))
vacancies = list(map(int,pairs['vacancy_id'].replace(vac2idx).to_numpy()))
preferences = pairs['action_type'].replace(action_weights).to_numpy()

In [11]:
pairs.head()

user_id,vacancy_id,action_type
str,str,i64
"""u_370846""","""v_697571""",1
"""u_461521""","""v_2514797""",2
"""u_332204""","""v_742810""",2
"""u_229153""","""v_1411424""",2
"""u_1128173""","""v_248154""",2


In [14]:
uv_mat = csr_matrix((preferences, (users, vacancies)))

# als_model = implicit.als.AlternatingLeastSquares(
#     factors=150,
#     random_state=RANDOM_STATE,
#     iterations=20,
#     calculate_training_loss=True,
#     regularization=0.001
# )
# als_model.fit(uv_mat)

In [15]:
# als_model.save("model.npz")
als_model = implicit.als.AlternatingLeastSquares(
    factors=150,
    random_state=RANDOM_STATE,
    iterations=20,
    calculate_training_loss=True,
    regularization=0.001
)
als_model = als_model.load("/kaggle/input/modell/model.npz")

In [21]:
# from IPython.display import FileLink
# FileLink(r'model.npz')

/kaggle/working/model.npz

In [22]:
als_model.user_factors.shape, als_model.item_factors.shape

((880985, 150), (1457882, 150))

In [16]:
test_users = test['user_id'].to_list()
test_vacancies = test.select(pl.col('vacancy_id').list.unique(maintain_order=True))['vacancy_id'].to_list()

In [17]:
predictions = []

for user, vacs in tqdm(zip(test_users, test_vacancies), total=len(test_users)):
    if user not in user2idx:
        predictions.append(vacs)
        continue
    cuser = user2idx[user]

    recommendations = als_model.recommend(cuser, uv_mat[cuser], N=N_PREDICTIONS)[0]
    recommendations = [idx2vac[cv] for cv in recommendations]
    predictions.append(recommendations)

100%|██████████| 83189/83189 [2:32:01<00:00,  9.12it/s]  


In [25]:
# test = test.with_columns(pl.lit(predictions).alias('predictions'))
# test.select(['user_id', 'session_id', 'predictions']).write_parquet('als_submission.pq')

In [26]:
# predictions = test.select(pl.col('vacancy_id').list.unique(maintain_order=True).list.tail(2))['vacancy_id'].to_list()
# test = test.with_columns(pl.lit(predictions).alias('predictions'))

# test.select(['user_id', 'session_id', 'predictions']).write_parquet('submission.pq')

In [18]:
df = pl.DataFrame({'predictions': [i for i in predictions]} )
test = test.with_columns(df)

In [19]:
test.select(['user_id', 'session_id', 'predictions']).write_parquet('submission.pq')

In [22]:
df.write_parquet('preds.pq')

In [23]:
from IPython.display import FileLink
FileLink(r'preds.pq')

/kaggle/working/preds.pq

In [14]:
load_preds = pl.read_parquet("/kaggle/input/first-preds-submisson/submission.pq")

""
str
"""v_285847"""
"""v_2107320"""
"""v_360867"""
"""v_844346"""
"""v_2386441"""
"""v_668611"""
"""v_2681278"""
"""v_1029119"""
"""v_1333276"""
